In [1]:
import pandas as pd

In [2]:
df = pd.read_pickle('dftest_sub2.p')
df = df[df['formula'].notna()]

In [3]:
print(df)

                                          formula        a         b  \
0                                           AuCu3   3.7480   3.74800   
1                                       Cr4O16Pb4   7.1167   7.41450   
2                                       N24Na24W8  13.7770  10.99900   
3                                      Ge4Nd20Sb8  12.2982   9.32220   
4                                      Cu8Er2Ga16   8.5910   8.59100   
...                                           ...      ...       ...   
71964    Ca0.02Fe0.12H7.84Mg0.07Mn0.99O12P2Zn1.82  10.6530  18.47810   
71965                               H32MgNa2O24S2   6.5590   6.62770   
71966  Al7.95B3Ca0.01F0.34H3Li1.05Na0.58O30.66Si6  15.8090  15.80900   
71967                                H4Mg10O18Si3  13.9696   5.09285   
71968                                H4Mg10O18Si3  13.9882   5.10006   

              c   alpha     beta    gamma      vol cellpressure        sg  \
0       3.74800  90.000   90.000   90.000    52.65       0

In [4]:
from pymatgen.core.composition import Composition
comp = Composition("SiO2")
comp.reduced_formula

'SiO2'

In [5]:
def returnReduced(string):
    comp = Composition(string)
    return comp.reduced_formula

In [6]:
df['reducedFormula'] = df['formula'].apply(lambda x: returnReduced(x))

C:\Users\Matthew\.conda\envs\MachineLearning\lib\site-packages\pymatgen-2022.5.26-py3.8-win-amd64.egg\pymatgen\core\periodic_table.py:210: UserWarning: No electronegativity for Ne. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  warnings.warn(
C:\Users\Matthew\.conda\envs\MachineLearning\lib\site-packages\pymatgen-2022.5.26-py3.8-win-amd64.egg\pymatgen\core\periodic_table.py:210: UserWarning: No electronegativity for Rf. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  warnings.warn(
C:\Users\Matthew\.conda\envs\MachineLearning\lib\site-packages\pymatgen-2022.5.26-py3.8-win-amd64.egg\pymatgen\core\periodic_table.py:210: UserWarning: No electronegativity for He. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  warnings.warn(
C:\Users\Matthew\.conda\envs\MachineLearning\lib\

In [7]:
print(df)

                                          formula        a         b  \
0                                           AuCu3   3.7480   3.74800   
1                                       Cr4O16Pb4   7.1167   7.41450   
2                                       N24Na24W8  13.7770  10.99900   
3                                      Ge4Nd20Sb8  12.2982   9.32220   
4                                      Cu8Er2Ga16   8.5910   8.59100   
...                                           ...      ...       ...   
71964    Ca0.02Fe0.12H7.84Mg0.07Mn0.99O12P2Zn1.82  10.6530  18.47810   
71965                               H32MgNa2O24S2   6.5590   6.62770   
71966  Al7.95B3Ca0.01F0.34H3Li1.05Na0.58O30.66Si6  15.8090  15.80900   
71967                                H4Mg10O18Si3  13.9696   5.09285   
71968                                H4Mg10O18Si3  13.9882   5.10006   

              c   alpha     beta    gamma      vol cellpressure        sg  \
0       3.74800  90.000   90.000   90.000    52.65       0

In [8]:
hashTableToIndexReptitions = {'NeEz': [-1, -2]}

In [9]:
hashTable = {'NeEZ': [[12, 14, 16, -1, -1, -1, -1 -1]]}
hashTableToIndexReptitionsA = {'NeEz': [-1, -2]}
tolerence = 0.05 # within 20%
def findPercentDiff(a, b):
    return (abs(a - b) / ((a + b)/2))

indexesToDrop = []
pairings = []
totalNum = 0
removed = 0
for idx, row in df.iterrows():
    if(row['reducedFormula'] == "SiO2"):
        indexesToDrop.append(idx)
    else:
        if(row['reducedFormula'] in hashTable):
            foundMatch = False
            indexInner = 0
            for array in hashTable[row['reducedFormula']]:
                if(findPercentDiff(row['a'], array[0]) <= tolerence and findPercentDiff(row['b'], array[1]) <= tolerence and findPercentDiff(row['c'], array[2]) <= tolerence and findPercentDiff(row['alpha'], array[3]) <= tolerence and findPercentDiff(row['beta'], array[4]) <= tolerence and findPercentDiff(row['gamma'], array[5]) <= tolerence):
                    indexesToDrop.append(idx)
                    removed += 1
                    if(row['reducedFormula'] not in hashTableToIndexReptitions):
                        hashTableToIndexReptitions[row['reducedFormula']] = [(idx, hashTable[row['reducedFormula']][indexInner][7])]
                    else:
                        hashTableToIndexReptitions[row['reducedFormula']].append((idx, hashTable[row['reducedFormula']][indexInner][7]))
                    foundMatch = True
                    break
                indexInner += 1
            if(not foundMatch):
                hashTable[row['reducedFormula']].append([row['a'], row['b'], row['c'], row['alpha'], row['beta'], row['gamma'], row['sgHall'], idx])
        else:
            hashTable[row['reducedFormula']] = [[row['a'], row['b'], row['c'], row['alpha'], row['beta'], row['gamma'], row['sgHall'], idx]]
        totalNum += 1

In [9]:
string = " "
for key in hashTableToIndexReptitions:
    if(key != 'NeEz' and len(hashTableToIndexReptitions[key]) > 1):
        string += "Reduced Formula: " + key + "\n"
        string += "    " + str(list(df.columns)) + "\n"
        for entry in hashTableToIndexReptitions[key]:
            first, second = entry
            string += "     Entry: " + str(list(df.loc[first])) + "\n"
            string += "     CONFLICT | Entry: " + str(list(df.loc[second])) + "\n"
        string += "------------------------------------\n\n"
print(len(hashTableToIndexReptitions))
f =open("5PercentRemovalsSiO2Removed3rd.txt",'a')
f.write(string)
f.close()

19909


In [43]:
string = "DIFFERENCES BETWEEN 20% and 2%: \n\n"
for key in hashTableToIndexReptitionsA:
    if(key != 'NeEz' and len(hashTableToIndexReptitionsA[key]) > 1) and key in hashTableToIndexReptitions:
        set1 = set(hashTableToIndexReptitionsA[key])
        set2 = set(hashTableToIndexReptitions[key])
        diffSet = set1.symmetric_difference(set2)
        if(len(diffSet) > 1):
            string += "Reduced Formula: " + key + "\n"
            string += "    " + str(list(df.columns)) + "\n"
            listDiff = list(diffSet)
            for entry in listDiff:
                first, second = entry
                string += "     Entry: " + str(list(df.loc[first])) + "\n"
                string += "     CONFLICT | Entry: " + str(list(df.loc[second])) + "\n"
            string += "------------------------------------\n\n"
print(string)
f =open("20to2PercentDiffPrettyNewer.txt",'a')
f.write(string)
f.close()

DIFFERENCES BETWEEN 20% and 2%: 

Reduced Formula: CeO2
    ['Unnamed: 0', 'formula', 'a', 'b', 'c', 'alpha', 'beta', 'gamma', 'vol', 'cellpressure', 'sg', 'sgHall', 'icsd_id', 'is_val', 'val', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'accepted', 'Not real', 'Is error', 'reducedFormula']
     Entry: [13482.0, 'Ce4O8', 5.5288, 5.5288, 5.5288, 90.0, 90.0, 90.0, 169.0023096, 0.0001, 'Fm-3m', 225.0, 'icsd_239398', nan, nan, nan, nan, nan, nan, nan, nan, nan, 'CeO2']
     CONFLICT | Entry: [258.0, 'Ce4O8', 5.4142, 5.4142, 5.4142, 90.0, 90.0, 90.0, 158.7094854, 0.0001, 'Fm-3m', 225.0, 'icsd_164226', nan, '0', nan, nan, nan, nan, 'CeO2', nan, nan, 'CeO2']
     Entry: [9874.0, 'Ce4O8', 5.6155, 5.6155, 5.6155, 90.0, 90.0, 90.0, 177.0782799, 0.0001, 'Fm-3m', 225.0, 'icsd_061607', nan, nan, nan, nan, nan, nan, nan, nan, nan, 'CeO2']
     CONFLICT | Entry: [258.0, 'Ce4O8', 5.4142, 5.4142, 5.4142, 90.0, 90.0, 90.0, 158.7094854, 0.0001, 'Fm-3m', 225.0, 'icsd_164226', nan, '0', nan

In [10]:
print(len(indexesToDrop))
print(removed)
print(totalNum)

3029
3029
71969


In [11]:
print(indexesToDrop)

[55326, 55327, 55328, 55331, 55332, 55333, 55334, 55338, 55339, 55341, 55345, 55346, 55347, 55348, 55349, 55354, 55355, 55356, 55357, 55358, 55359, 55360, 55361, 55364, 55365, 55366, 55369, 55370, 55371, 55373, 55374, 55375, 55378, 55379, 55380, 55381, 55382, 55384, 55385, 55386, 55387, 55388, 55389, 55390, 55391, 55392, 55398, 55420, 55423, 55426, 55429, 55430, 55431, 55433, 55435, 55437, 55438, 55439, 55441, 55445, 55446, 55447, 55448, 55449, 55457, 55458, 55459, 55461, 55462, 55463, 55465, 55466, 55467, 55468, 55469, 55470, 55471, 55472, 55473, 55474, 55475, 55476, 55477, 55478, 55479, 55480, 55482, 55483, 55484, 55485, 55486, 55487, 55488, 55490, 55491, 55492, 55493, 55494, 55495, 55496, 55497, 55498, 55500, 55501, 55502, 55503, 55504, 55505, 55506, 55507, 55508, 55509, 55510, 55511, 55512, 55513, 55515, 55516, 55524, 55525, 55526, 55527, 55528, 55529, 55530, 55533, 55534, 55535, 55536, 55537, 55538, 55539, 55540, 55541, 55542, 55543, 55544, 55545, 55546, 55547, 55548, 55549, 55550

In [24]:
string = ""
for entry in pairings:
    string += str(entry) + "\n"
print(string)
f =open("2PercentRemoval.txt",'a')
f.write(string)
f.close()

In [12]:
removed2PercentDF = df.drop(index = indexesToDrop)

In [13]:
print(removed2PercentDF)

                                          formula        a        b         c  \
0                                           AuCu3   3.7480   3.7480   3.74800   
1                                       Cr4O16Pb4   7.1167   7.4145   6.75660   
2                                       N24Na24W8  13.7770  10.9990   6.40100   
3                                      Ge4Nd20Sb8  12.2982   9.3222   8.15990   
4                                      Cu8Er2Ga16   8.5910   8.5910   5.16300   
...                                           ...      ...      ...       ...   
71962                              C2H30Co2N8O13S   7.4550  10.6090  23.62700   
71963                                Al3H22O24PS2  10.3410  10.9600  11.14460   
71964    Ca0.02Fe0.12H7.84Mg0.07Mn0.99O12P2Zn1.82  10.6530  18.4781   5.05845   
71965                               H32MgNa2O24S2   6.5590   6.6277  14.44100   
71966  Al7.95B3Ca0.01F0.34H3Li1.05Na0.58O30.66Si6  15.8090  15.8090   7.08900   

        alpha     beta    g

In [14]:
removed2PercentDF.to_csv('5PercentDFSiO2Removed4th.csv')

C:\Users\Matthew\.conda\envs\MachineLearning\lib\site-packages\pymatgen-2022.5.26-py3.8-win-amd64.egg\pymatgen\core\periodic_table.py:210: UserWarning: No electronegativity for Ne. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  warnings.warn(
C:\Users\Matthew\.conda\envs\MachineLearning\lib\site-packages\pymatgen-2022.5.26-py3.8-win-amd64.egg\pymatgen\core\periodic_table.py:210: UserWarning: No electronegativity for Ar. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  warnings.warn(
